In [2]:
from IPython.display import HTML, display

In [3]:
def calculateNodes(stake, rewardrate, targetDays, startNodes):
    currentNode = startNodes
    days = 0
    while (True):
        cycleTime = stake / rewardrate / currentNode
        days += cycleTime
        if (days > targetDays):
            break
        currentNode += 1
    return { "endNodes": currentNode, "newNodes": currentNode -  startNodes}

In [4]:
def nice(number):
    return format(int(number), ',f')

In [15]:
def calculateYear(year, startNodes, params):
    result = calculateNodes(
        params["initial"]["stake"] * params["years"][year]["growthFraction"],
        params["years"][year]["rewardrate"], 
        params["years"][year].get("days", 365), 
        startNodes)
    
    yearlyRewards = result["newNodes"] * params["initial"]["stake"] * params["years"][year]["growthFraction"]
    nodeStake = result["newNodes"] * params["initial"]["stake"]
    payout = yearlyRewards - nodeStake
    totalStake = params["initial"]["stake"] * result["endNodes"]
    
    return [
        year, 
        result["newNodes"], 
        result["endNodes"], 
        nice(yearlyRewards),
        #nice(nodeStake),
        #nice(totalStake),
        #nice(payout),
        nice((totalStake - (params["initial"]["stake"] * params["initial"]["startNodes"])) * params["initial"]["uprShare"]),
        nice((totalStake - (params["initial"]["stake"] * params["initial"]["startNodes"])) * params["initial"]["uprShare"] * params["years"][year]["price"]),
        nice(yearlyRewards * params["years"][year]["dividends"] * params["initial"]["uprShare"]),
        nice(yearlyRewards * params["years"][year]["dividends"] * params["initial"]["uprShare"] * params["years"][year]["price"]),
        nice((totalStake - (params["initial"]["stake"] * params["initial"]["startNodes"])) * params["initial"]["investorShare"]),
        nice((totalStake - (params["initial"]["stake"] * params["initial"]["startNodes"])) * params["initial"]["investorShare"] * params["years"][year]["price"]),
        nice(yearlyRewards * params["years"][year]["dividends"] * params["initial"]["investorShare"]),
        nice(yearlyRewards * params["years"][year]["dividends"] * params["initial"]["investorShare"] * params["years"][year]["price"])
    ]

In [21]:
def calculateYears(params):
    poktsteps = []
    data = [[
        'year', 
        'newNodes', 
        'endNodes', 
        'yearlyRewards', 
        #'nodeStakeRewards', 
        #'totalStake', 
        #'payout',
        'upr stake POKT',
        'upr stake USD',
        'upr dividends POKT',
        'upr dividends USD',
        'investor stake POKT',
        'investor stake USD',
        'investor dividends POKT',
        'investor dividends USD'
        ]]
    
    for year in params["years"].keys():
        startNodes = data[-1][2] if len(data) > 1 else params["initial"]["startNodes"]
        data.append(calculateYear(year, startNodes, params))
        
    display(HTML(
        '<table><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
    ))

In [34]:
calculateYears({
    "initial": {
        "stake": 15100,
        "startNodes": 750,
        "investorShare": .15,
        "uprShare": .85,
    },
    "years": {
        2022: { "growthFraction": 100/100, "dividends": 0, "rewardrate": 20, "price": 0.10, "days": 90},
        2023: { "growthFraction": 100/100, "dividends": 0, "rewardrate": 20, "price": 0.10},
        2024: { "growthFraction": 100/100, "dividends": 0, "rewardrate": 20, "price": 0.10},
        2025: { "growthFraction": 100/70, "dividends": .3, "rewardrate": 20, "price": 0.10},
        2026: { "growthFraction": 100/10, "dividends": .9, "rewardrate": 20, "price": 0.10},
        2027: { "growthFraction": 100/10, "dividends": .9, "rewardrate": 20, "price": 0.10},
        2028: { "growthFraction": 100/10, "dividends": .9, "rewardrate": 20, "price": 0.10},
    }
})

year,newNodes,endNodes,yearlyRewards,upr stake POKT,upr stake USD,upr dividends POKT,upr dividends USD,investor stake POKT,investor stake USD,investor dividends POKT,investor dividends USD
2022,94,844,"1,419,400.000000","1,206,490.000000","120,649.000000",0.000000,0.000000,"212,910.000000","21,291.000000",0.000000,0.000000
2023,524,1368,"7,912,400.000000","7,932,030.000000","793,203.000000",0.000000,0.000000,"1,399,770.000000","139,977.000000",0.000000,0.000000
2024,850,2218,"12,835,000.000000","18,841,780.000000","1,884,178.000000",0.000000,0.000000,"3,325,020.000000","332,502.000000",0.000000,0.000000
2025,893,3111,"19,263,285.000000","30,303,435.000000","3,030,343.000000","4,912,137.000000","491,213.000000","5,347,665.000000","534,766.000000","866,847.000000","86,684.000000"
2026,154,3265,"23,254,000.000000","32,280,025.000000","3,228,002.000000","17,789,310.000000","1,778,931.000000","5,696,475.000000","569,647.000000","3,139,290.000000","313,929.000000"
2027,161,3426,"24,311,000.000000","34,346,460.000000","3,434,646.000000","18,597,915.000000","1,859,791.000000","6,061,140.000000","606,114.000000","3,281,985.000000","328,198.000000"
2028,169,3595,"25,519,000.000000","36,515,575.000000","3,651,557.000000","19,522,035.000000","1,952,203.000000","6,443,925.000000","644,392.000000","3,445,065.000000","344,506.000000"
